In [1]:
using Revise
using Plots
using FFTW
using Compat
using WAV
using DSP
using Base64
using ForneyLab
using LinearAlgebra
using ProgressMeter

In [20]:
include("../extensions/complex_gaussian.jl")
include("../extensions/hgf.jl")

In [74]:
# generate data
import Distributions: Normal, MvNormal, MixtureModel, Dirichlet
N = 1
nr_freqs = 2
n_samples = N

μ_ξi = 1.0*collect(-4:2:4)
Σ_ξ = 1e-4*diagm(ones(nr_freqs))
Σ_meas = 1e-10*diagm(ones(nr_freqs))

ξ_samples = Array{Float64,2}(undef, N, nr_freqs)
X_samples = Array{Complex{Float64},2}(undef, N, nr_freqs)
y_samples = Array{Complex{Float64},2}(undef, N, nr_freqs)

for n = 1:N

#     sample_ξ = rand(MvNormal(μ_ξ, Σ_ξ))
#     sample_X = rand(MvNormal(zeros(nr_freqs), real.((exp.(0.5*sample_ξ))))) + 1im*rand(MvNormal(zeros(nr_freqs), real.((exp.(0.5*sample_ξ)))))
#     sample_y = rand(MvNormal(real(sample_X), 0.5*real.(Σ_meas))) + 1im*rand(MvNormal(imag(sample_X), 0.5*real.(Σ_meas)))
    
#     ξ_samples[n,:] = sample_ξ
#     X_samples[n,:] = sample_X
#     y_samples[n,:] = sample_y
    for k = 1:nr_freqs
        sample_ξ = rand(Normal(μ_ξi[k], sqrt(Σ_ξ[k,k])))
        sample_X = rand(Normal(0, sqrt(0.5*exp(sample_ξ)))) + 1im*rand(Normal(0, sqrt(0.5*exp(sample_ξ))))
        sample_y = rand(Normal(real(sample_X), sqrt(0.5*Σ_meas[k,k]))) + 1im*rand(Normal(imag(sample_X), sqrt(0.5*Σ_meas[k,k])))

        ξ_samples[n,k] = sample_ξ
        X_samples[n,k] = sample_X
        y_samples[n,k] = sample_y
    end
end

t = collect(1:N)
Y = y_samples

1×2 Array{Complex{Float64},2}:
 -0.0471331+0.0937773im  -0.23022-0.0672051im

# Building graph

In [68]:
nr_freqs = dimension

fg = FactorGraph()
α = 0.5
# Specify generative model
@RV _pi ~ ForneyLab.Dirichlet(α*ones(nr_freqs))
@RV m_1 ~ GaussianMeanVariance(zeros(nr_freqs), 100*diagm(ones(nr_freqs)))
@RV w_1 ~ Wishart(diagm(ones(nr_freqs)), 5.0)
@RV m_2 ~ GaussianMeanVariance(zeros(nr_freqs), 100*diagm(ones(nr_freqs)))
@RV w_2 ~ Wishart(diagm(ones(nr_freqs)), 5.0)

z = Vector{Variable}(undef, n_samples)
ξ = Vector{Variable}(undef, n_samples)
X = Vector{Variable}(undef, n_samples)
y = Vector{Variable}(undef, n_samples)
for i in 1:n_samples
    @RV z[i] ~ Categorical(_pi)
    @RV ξ[i] ~ GaussianMixture(z[i], m_1, w_1, m_2, w_2)
    # HGF
    @RV X[i] ~ HGF(ξ[i])

    # observation model
    @RV y[i] ~ ComplexNormal(X[i], 1e-10*diagm(ones(nr_freqs)).+0im, mat(0.0+0.0im))
    
    
    placeholder(y[i], :y, index=i, dims=(nr_freqs,))
end
# draw graph
ForneyLab.draw(fg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 18080429588601912121 -->
 
 18080429588601912121 
 
 Dir 
 dirichlet_1 
 
<!-- 621077407988664570 -->
 
 621077407988664570 
 
 clamp_1 
 
<!-- 18080429588601912121--621077407988664570 -->
 
 18080429588601912121--621077407988664570 
 
 clamp_1 
 1 out 
 2 a 
 
<!-- 9351585832680325924 -->
 
 9351585832680325924 
 
 clamp_7 
 
<!-- 8716014573168625084 -->
 
 8716014573168625084 
 
 HGF 
 hgf_1 
 
<!-- 12882985716769891414 -->
 
 12882985716769891414 
 
 GM 
 gaussianmixture_1 
 
<!-- 8716014573168625084--12882985716769891414 -->
 
 8716014573168625084--12882985716769891414 
 
 ξ_1 
 1 out 
 2 ξ 
 
<!-- 9947607146227518956 -->
 
 9947607146227518956 
 
 clamp_8 
 
<!-- 2380556556703155491 -->
 
 2380556556703155491 
 
 clamp_4 
 
<!-- 16528909271086645245 -->
 
 16528909271086645245 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 12882985716769891414--16528909271086645245 -->
 
 12882985716769891414--16528909271086645245 
 
 m_2 
 1 out 
 5 m2 
 
<!-- 11268296456772929567 -->
 
 11268296456772929567 
 
 W 
 wishart_2 
 
<!-- 12882985716769891414--11268296456772929567 -->
 
 12882985716769891414--11268296456772929567 
 
 w_2 
 1 out 
 6 w2 
 
<!-- 14366408256047734447 -->
 
 14366408256047734447 
 
 W 
 wishart_1 
 
<!-- 12882985716769891414--14366408256047734447 -->
 
 12882985716769891414--14366408256047734447 
 
 w_1 
 1 out 
 4 w1 
 
<!-- 2294880291768833558 -->
 
 2294880291768833558 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 12882985716769891414--2294880291768833558 -->
 
 12882985716769891414--2294880291768833558 
 
 m_1 
 1 out 
 3 m1 
 
<!-- 16973507398096972165 -->
 
 16973507398096972165 
 
 Cat 
 categorical_1 
 
<!-- 12882985716769891414--16973507398096972165 -->
 
 12882985716769891414--16973507398096972165 
 
 z_1 
 1 out 
 2 z 
 
<!-- 16528909271086645245--9351585832680325924 -->
 
 16528909271086645245--9351585832680325924 
 
 clamp_7 
 1 out 
 3 v 
 
<!-- 6737949355396232922 -->
 
 6737949355396232922 
 
 clamp_6 
 
<!-- 16528909271086645245--6737949355396232922 -->
 
 16528909271086645245--6737949355396232922 
 
 clamp_6 
 1 out 
 2 m 
 
<!-- 13011104526291389745 -->
 
 13011104526291389745 
 
 clamp_5 
 
<!-- 5956014648573380156 -->
 
 5956014648573380156 
 
 clamp_9 
 
<!-- 6727221768884708959 -->
 
 6727221768884708959 
 
 ComplexNormal 
 complexnormal_1 
 
<!-- 6727221768884708959--8716014573168625084 -->
 
 6727221768884708959--8716014573168625084 
 
 X_1 
 1 X 
 2 μ 
 
<!-- 13539388162578894275 -->
 
 13539388162578894275 
 
 clamp_10 
 
<!-- 6727221768884708959--13539388162578894275 -->
 
 6727221768884708959--13539388162578894275 
 
 clamp_10 
 1 out 
 3 Γ 
 
<!-- 13684146036743433361 -->
 
 13684146036743433361 
 
 clamp_11 
 
<!-- 6727221768884708959--13684146036743433361 -->
 
 6727221768884708959--13684146036743433361 
 
 clamp_11 
 1 out 
 4 C 
 
<!-- 11268296456772929567--9947607146227518956 -->
 
 11268296456772929567--9947607146227518956 
 
 clamp_8 
 1 out 
 2 v 
 
<!-- 11268296456772929567--5956014648573380156 -->
 
 11268296456772929567--5956014648573380156 
 
 clamp_9 
 1 out 
 3 nu 
 
<!-- 5215521784856882956 -->
 
 5215521784856882956 
 
 clamp_2 
 
<!-- 7816657981060047785 -->
 
 7816657981060047785 
 
 placeholder_y_1 
 
<!-- 7816657981060047785--6727221768884708959 -->
 
 7816657981060047785--6727221768884708959 
 
 y_1 
 1 out 
 1 out 
 
<!-- 17652001903655194299 -->
 
 17652001903655194299 
 
 clamp_3 
 
<!-- 14366408256047734447--2380556556703155491 -->
 
 14366408256047734447--2380556556703155491 
 
 clamp_4 
 1 out 
 2 v 
 
<!-- 14366408256047734447--13011104526291389745 -->
 
 14366408256047734447--13011104526291389745 
 
 clamp_5 
 1 out 
 3 nu 
 
<!-- 2294880291768833558--5215521784856882956 -->
 
 2294880291768833558--5215521784856882

In [82]:
# Build the algorithm
algo = variationalAlgorithm(_pi, m_1, w_1, m_2, w_2, z, X, ξ, ids=[:PI :M1 :W1 :M2 :W2 :Z :X :Ξ])

# Generate source code
source_code = algorithmSourceCode(algo);

# Load algorithm
eval(Meta.parse(source_code));

In [83]:
println(source_code)

begin

function stepZ!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBCategoricalOut(nothing, marginals[:_pi])
messages[2] = ruleVBGaussianMixtureZBer(marginals[:ξ_1], nothing, marginals[:m_1], marginals[:w_1], marginals[:m_2], marginals[:w_2])

marginals[:z_1] = messages[1].dist * messages[2].dist

return marginals

end

function stepΞ!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGaussianMixtureOut(nothing, marginals[:z_1], marginals[:m_1], marginals[:w_1], marginals[:m_2], marginals[:w_2])
messages[2] = ruleVariationalHGFIn1PN(marginals[:X_1], nothing)

marginals[:ξ_1] = messages[1].dist * messages[2].dist

return marginals

end

function stepM1!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGaussianMeanVarianceOut(nothing, ProbabilityDistribution(Multivariate, PointMass, m=[0.0, 0.0]), Probabilit

In [77]:
data = Dict(:y => Y)

# Prepare posterior factors
marginals = Dict(:_pi => vague(ForneyLab.Dirichlet, nr_freqs),
                 :m_1 => ProbabilityDistribution(Multivariate, GaussianMeanVariance, m=-1.0*ones(nr_freqs), v=1e4*diagm(ones(nr_freqs))),
                 :w_1 => vague(Wishart, nr_freqs),
                 :m_2 => ProbabilityDistribution(Multivariate, GaussianMeanVariance, m=ones(nr_freqs), v=1e4*diagm(ones(nr_freqs))),
                 :w_2 => vague(Wishart, nr_freqs))
for i in 1:n_samples
    marginals[:z_*i] = vague(Categorical)
    marginals[:X_*i] = ProbabilityDistribution(Multivariate, ComplexNormal, μ=zeros(nr_freqs) .+ 0.0im, Γ=1e-10*diagm(ones(nr_freqs)).+0im, C=mat(0.0+0.0im));
    marginals[:ξ_*i] = ProbabilityDistribution(ForneyLab.Multivariate, GaussianMeanVariance, m=zeros(nr_freqs), v=diagm(ones(nr_freqs)))
end



In [81]:
# Execute algorithm
n_its = 10
@showprogress for i in 1:n_its
    stepX!(data, marginals)
    stepΞ!(data, marginals)
    stepZ!(data, marginals)
    stepPI!(data, marginals)
    stepM1!(data, marginals)
    stepW1!(data, marginals)
    stepM2!(data, marginals)
    stepW2!(data, marginals)   
end

TypeError: TypeError: in keyword argument m, expected Array{T,1} where T, got Array{Complex{Float64},2}